# Task 1:

# Plant Seedling Classification

# 0. Intro

**Task type:** Classification

**ML algorithms used:** Convolutional neural network

**Other features:** Visualizing the filters used by the CNN

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle


import cv2

# 1. Image importing

In [ ]:
image_size = 256
batch_size = 32

**I will load the images using ImageDataGeneretor class and explicitly indicate the needed parameters (rescaling, flipping, validation splitting.**

In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    #rotation_range=20, # You can uncomment these parameters to make you generator rotate & flip the images to put the train model in stricter conditions.
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

**Train set**

In [ ]:
train_gen = idg.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                    target_size=(image_size, image_size),
                                                    subset='training',
                                                    class_mode='categorical',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=1
                                                )

**Validation set**

In [ ]:
val_gen = idg.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                   target_size=(image_size, image_size),                                                   
                                                   subset='validation',
                                                   class_mode='categorical',
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   seed=1
                                                )

In [ ]:
unique, counts = np.unique(train_gen.classes, return_counts=True)
dict1 = dict(zip(train_gen.class_indices, counts))

keys = dict1.keys()
values = dict1.values()

plt.xticks(rotation='vertical')
bar = plt.bar(keys, values)

**The train dataset is quite balanced.**

**Let's visualize a portion of images to make sure they're correctly loaded.**

In [ ]:
x,y = next(train_gen)

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

def show_grid(image_list, nrows, ncols, label_list=None, show_labels=False, figsize=(10,10)):

    fig = plt.figure(None, figsize,frameon=False)
    grid = ImageGrid(fig, 111, 
                     nrows_ncols=(nrows, ncols),  
                     axes_pad=0.2, 
                     share_all=True,
                     )
    for i in range(nrows*ncols):
        ax = grid[i]
        ax.imshow(image_list[i],cmap='Greys_r')
        ax.axis('off')

In [ ]:
show_grid(x,2,4,show_labels=True,figsize=(10,10))

**The images are nicely loaded and do not have any rotation and distortion.**

# 2. Model building

**Quick description, before heading on:**

**Model type:** Sequential

**Layers used:**

    0. InputLayer
    1. Conv2D (64, 128 filters)
    2. MaxPool2D
    3. GlobalMaxPool2D
    4. Batch Normalization
    5. Flatten
    6. Dropout
    6. Dense
    
**Input size:** 256 x 256 x 3 (size x colors)

**Pool size:** 2 x 2 (for MaxPool2D)

**Kernel size:** 3 x 3 (for Conv2D)

In [ ]:
#NN
modelNN1 = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(12, activation='softmax')
    
])

In [ ]:
modelNN1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelNN1.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('plant_classifierNN.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history = modelNN1.fit(train_gen,
          epochs=20, # Increase number of epochs if you have sufficient hardware
          steps_per_epoch= 3803//batch_size,  # Number of train images // batch_size
          validation_data=val_gen,
          validation_steps = 947//batch_size, # Number of val images // batch_size
          callbacks = [checkpoint],
          verbose = 1
)

In [ ]:
#CNN
modelCNN1 = tf.keras.models.Sequential()

# Input layer
# Can be omitted, you can specify the input_shape in other layers
modelCNN1.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))

# Here we add a 2D Convolution layer
# Check https://keras.io/api/layers/convolution_layers/convolution2d/ for more info
modelCNN1.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

# Max Pool layer 
# It downsmaples the input representetion within the pool_size size
modelCNN1.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
# The layer normalizes its output using the mean and standard deviation of the current batch of inputs.
modelCNN1.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
modelCNN1.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
modelCNN1.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
modelCNN1.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
modelCNN1.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
modelCNN1.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
modelCNN1.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
modelCNN1.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
modelCNN1.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Global Max Pool layer
modelCNN1.add(tf.keras.layers.GlobalMaxPool2D())

# Dense Layers after flattening the data
modelCNN1.add(tf.keras.layers.Flatten())

modelCNN1.add(tf.keras.layers.Dense(128, activation='relu'))

# Dropout
# is used to nullify the outputs that are very close to zero and thus can cause overfitting.
modelCNN1.add(tf.keras.layers.Dropout(0.2))
modelCNN1.add(tf.keras.layers.Dense(64, activation='relu'))

# Normalization layer
modelCNN1.add(tf.keras.layers.BatchNormalization())

#Add Output Layer
modelCNN1.add(tf.keras.layers.Dense(12, activation='softmax')) # = 12 predicted classes

In [ ]:
modelCNN1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modelCNN1.summary()

In [ ]:
# You can save the best model to the checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('plant_classifier.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history = modelCNN1.fit(train_gen,
          epochs=20, # Increase number of epochs if you have sufficient hardware
          steps_per_epoch= 3803//batch_size,  # Number of train images // batch_size
          validation_data=val_gen,
          validation_steps = 947//batch_size, # Number of val images // batch_size
          callbacks = [checkpoint],
          verbose = 1
)

**Learning curves vs epoch graph**

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(list(range(1,21)))
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
weigh= modelCNN1.get_weights()
pklfile= "modelweights.pkl"
try:
    fpkl= open(pklfile, 'wb')    #Python 3     
    pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
    fpkl.close()
except:
    fpkl= open(pklfile, 'w')    #Python 2      
    pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
    fpkl.close()

# Visualizing

**Here we will check the predictions made by our model as well as visualize how the model filter one of the images taken from the dataset.**

**Let's first check the prediction power.**

In [ ]:
maize = cv2.imread('../input/prediction/Predict.png')

In [ ]:
plt.imshow(maize)

**We need to preprocess the image before passing it on to the model: resize + expand_dims.**

In [ ]:
maize = cv2.resize(maize, (256,256))

In [ ]:
maize_batch = np.expand_dims(maize, axis=0)

In [ ]:
conv_maize = modelCNN1.predict(maize_batch)

In [ ]:
# conv_maize.shape
ind=np.argmax(conv_maize,axis=1)

In [ ]:
print(list(train_gen.class_indices.keys())[ind[0]])

# 4. Conclusion

We have built a **CNN-model** to predict the class of a plant, which works quite well. (Increasing **number of epochs** and/or **adding layers** to a model can even increase the performance.

**CNN + Maxpooling + Global pooling + Dense** is a good combination for image classification.


# Task 2:


Neural Networks (NN), or more precisely Artificial Neural Networks (ANN), is a class of Machine Learning algorithms that recently received a lot of attention (again!) due to the availability of Big Data and fast computing facilities (most of Deep Learning algorithms are essentially different variations of ANN).

The class of ANN covers several architectures including Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN) eg LSTM and GRU, Autoencoders, and Deep Belief Networks. Therefore, CNN is just one kind of ANN.

Generally speaking, an ANN is a collection of connected and tunable units (a.k.a. nodes, neurons, and artificial neurons) which can pass a signal (usually a real-valued number) from a unit to another. The number of (layers of) units, their types, and the way they are connected to each other is called the network architecture.

A CNN, in specific, has one or more layers of convolution units. A convolution unit receives its input from multiple units from the previous layer which together create a proximity. Therefore, the input units (that form a small neighborhood) share their weights.

The convolution units (as well as pooling units) are especially beneficial as:

They reduce the number of units in the network (since they are many-to-one mappings). This means, there are fewer parameters to learn which reduces the chance of overfitting as the model would be less complex than a fully connected network.
They consider the context/shared information in the small neighborhoods. This future is very important in many applications such as image, video, text, and speech processing/mining as the neighboring inputs (eg pixels, frames, words, etc) usually carry related information.

Also from our outputs we can see the that CNN outperfromed NN with a very wide gap of accuracy.

# Task 3:

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
height_shift_range=0.1,shear_range=0.15, 
zoom_range=0.1,channel_shift_range = 10, horizontal_flip=True)

In [ ]:
import os
import matplotlib.pyplot as plt

In [ ]:
images=np.array([1,2,3])
len(images)

In [ ]:
image_path = '../input/carsdata'
images=np.array([])

files=os.listdir(image_path)
for file in files:
    path=image_path+'/'+file
    img=plt.imread(path)
    img=cv2.resize(img,(224,224))
    if len(images)==0:
        images = np.expand_dims(img, 0)
    images=np.append(images,[img],axis=0)
images.shape
# image = np.expand_dims(ndimage.imread(image_path), 0)

In [ ]:
os.mkdir('newcardata')

In [ ]:
save_here = './newcardata'

In [ ]:
datagen.fit(images)

In [ ]:
for x, val in zip(datagen.flow(images,                    #image we chose
        save_to_dir=save_here,     #this is where we figure out where to save
         save_prefix='aug',        # it will save the images as 'aug_0912' some number for every new augmented image
        save_format='png'),range(10)) :     # here we define a range because we want 10 augmented images otherwise it will keep looping forever I think
    pass

In [ ]:
fig = plt.figure(1, (14, 14))

image_path = './newcardata'
files=os.listdir(image_path)


k = 0
for file in files:
    path=image_path+'/'+file
    img=plt.imread(path)
    img=cv2.resize(img,(224,224))

    
    k += 1
    if k==50:
        break
    ax = plt.subplot(7, 7, k)
    ax.imshow(img, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

# **So we succesfully created the augmented images from the cars images.**

# Task 4:

In [ ]:
!pip install tflearn
import tflearn.datasets.oxflower17 as oxflower17
X, Y = oxflower17.load_data(one_hot=True)

In [ ]:
Y.shape

In [ ]:
#NN
modelNN2 = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224,224,3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(17, activation='softmax')
    
])

In [ ]:
show_grid(X,2,4,show_labels=True,figsize=(10,10))

In [ ]:
modelNN2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelNN2.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('flower_classifierNN.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_acc', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history=modelNN2.fit(X, Y, batch_size=64, epochs=60, verbose=1, validation_split=0.2, shuffle=True,callbacks = [checkpoint])

In [ ]:
#CNN
modelCNN2 = tf.keras.models.Sequential()

# Input layer
# Can be omitted, you can specify the input_shape in other layers
modelCNN2.add(tf.keras.layers.InputLayer(input_shape=(224,224,3,)))

# Here we add a 2D Convolution layer
# Check https://keras.io/api/layers/convolution_layers/convolution2d/ for more info
modelCNN2.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

# Max Pool layer 
# It downsmaples the input representetion within the pool_size size
modelCNN2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
# The layer normalizes its output using the mean and standard deviation of the current batch of inputs.
modelCNN2.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
modelCNN2.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
modelCNN2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
modelCNN2.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
modelCNN2.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
modelCNN2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Normalization layer
modelCNN2.add(tf.keras.layers.BatchNormalization())

# 2D Convolution layer
modelCNN2.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))

# Max Pool layer 
modelCNN2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))

# Global Max Pool layer
modelCNN2.add(tf.keras.layers.GlobalMaxPool2D())

# Dense Layers after flattening the data
modelCNN2.add(tf.keras.layers.Flatten())

modelCNN2.add(tf.keras.layers.Dense(128, activation='relu'))

# Dropout
# is used to nullify the outputs that are very close to zero and thus can cause overfitting.
modelCNN2.add(tf.keras.layers.Dropout(0.2))
modelCNN2.add(tf.keras.layers.Dense(64, activation='relu'))

# Normalization layer
modelCNN2.add(tf.keras.layers.BatchNormalization())

#Add Output Layer
modelCNN2.add(tf.keras.layers.Dense(17, activation='softmax')) # = 12 predicted classes

In [ ]:
modelCNN2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelCNN2.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('flower_classifierNN.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_acc', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history=modelCNN2.fit(X, Y, batch_size=64, epochs=70, verbose=1, validation_split=0.2, shuffle=True,callbacks = [checkpoint])

In [ ]:
#CNN with Transfer Learning
base = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
# base = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))

base.trainable = False

# for i in range(len(resnet.layers)-8):
#     resnet.layers[i].trainable = False
#     print(resnet.layers[i])
    
model = tf.keras.Sequential([
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(128,name="dense100",activation="relu"),
    tf.keras.layers.Dropout(0.3),


    
    tf.keras.layers.Dense(17,activation="softmax")
])

In [ ]:
base.trainable = True

In [ ]:
for i in range(16):
    base.layers[i].trainable = False
#     print(resnet.layers[i])
base.summary()
# base.layers[15].trainable = True
# base.layers[16].trainable = True
# base.layers[17].trainable = True

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('flower_classifierNN.h5', #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_acc', 
                                                    mode='max', 
                                                    verbose = 1)

In [ ]:
history=model.fit(X, Y, batch_size=64, epochs=30, verbose=1, validation_split=0.2, shuffle=True,callbacks = [checkpoint])

So as we can see CNN with transfer learning gave a val_acc of 93.75%
while CNN had 86.09% and NN had 48.53% and hence we can say that
CNN with transfer learning performs a lot better than any of the rest.